In [3]:
!pip install tensorflow scikit-learn scikeras

INFO: pip is looking at multiple versions of scikeras to determine which version is compatible with other requirements. This could take a while.


In [5]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV

# Load the CIFAR-10 dataset
(x_train, y_train), (x_val, y_val) = cifar10.load_data()

# Use only a subset of the data for faster computation
x_train, y_train = x_train[:5000], y_train[:5000]
x_val, y_val = x_val[:1000], y_val[:1000]

# Normalize the data
x_train = x_train.astype('float32') / 255
x_val = x_val.astype('float32') / 255

# Convert labels to one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_val = tf.keras.utils.to_categorical(y_val, 10)

# Define the function to create the model
def create_model(optimizer='adam', learning_rate=0.01):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(10, activation='softmax'))

    # Compile the model
    if optimizer == 'adam':
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)

    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Create the KerasClassifier
model = KerasClassifier(model=create_model, verbose=0, optimizer='adam', learning_rate=0.01, epochs=5, batch_size=32)

# Define the parameter grid for GridSearchCV
param_grid = {
    'optimizer': ['adam', 'sgd'],
    'learning_rate': [0.001, 0.01],
    'epochs': [5],
    'batch_size': [32]
}

# Perform GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(x_train, y_train)

# Print the results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
for mean, std, params in zip(grid_result.cv_results_['mean_test_score'], grid_result.cv_results_['std_test_score'], grid_result.cv_results_['params']):
    print(f"Mean: {mean:.4f}, Std: {std:.4f} with: {params}")

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


Best: 0.5023997601440097 using {'batch_size': 32, 'epochs': 5, 'learning_rate': 0.001, 'optimizer': 'adam'}
Mean: 0.5024, Std: 0.0042 with: {'batch_size': 32, 'epochs': 5, 'learning_rate': 0.001, 'optimizer': 'adam'}
Mean: 0.4748, Std: 0.0143 with: {'batch_size': 32, 'epochs': 5, 'learning_rate': 0.001, 'optimizer': 'sgd'}
Mean: 0.2398, Std: 0.1139 with: {'batch_size': 32, 'epochs': 5, 'learning_rate': 0.01, 'optimizer': 'adam'}
Mean: 0.2558, Std: 0.1197 with: {'batch_size': 32, 'epochs': 5, 'learning_rate': 0.01, 'optimizer': 'sgd'}
